<a href="https://colab.research.google.com/github/Ivan-KKH/APAI4011_GP/blob/master/predicting_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#from google.colab import files
import numpy as np
import pandas as pd
import io
import re
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Bidirectional
import warnings
import random
warnings.filterwarnings("ignore")

2023-05-07 00:25:35.169175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 00:25:35.217447: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 00:25:35.218358: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 00:25:35.799262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#uploaded = files.upload()

In [4]:
#read csv file
#df = pd.read_csv(io.BytesIO(uploaded['amazon.csv']))
df = pd.read_csv('amazon.csv')
#dfReview for review title and review content
dfText = df[['product_name','about_product','category']]

In [5]:
for column in range(2):
  dfText.iloc[:,column] = dfText.iloc[:,column].apply(lambda x: x.lower())
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub('[^a-zA-z0-9\s]'," ",x)))
dfText['category'] = dfText['category'].apply(lambda x: x.split('|',1)[0])

#combine product_name and about_product
dfText['combined'] = dfText['product_name'] + ' ' + dfText['about_product']

In [6]:
#tokenize, build vocab, and add paddings on product_name
tokenizer = Tokenizer(num_words=2000,split=' ')
tokenizer.fit_on_texts(dfText['combined'].values)
text = tokenizer.texts_to_sequences(dfText['combined'].values)
text = pad_sequences(text)

#category
category_list = []
for categ in dfText['category']:
  if categ not in category_list:
    category_list.append(categ)
print(f'category list: {category_list}')
category = pd.get_dummies(dfText['category'], columns=category_list).values
category_list = ['Car & Motorbike', 'Computers & Accessories', 'Electronics', 'Health & Personal Care', 'Home & Kitchen', 'Home Improvement', 'Musical Instruments', 'Office Products', 'Toys & Games']

print(f'shape of text: {text.shape}')
print(f'shape of category: {category.shape}')


category list: ['Computers&Accessories', 'Electronics', 'MusicalInstruments', 'OfficeProducts', 'Home&Kitchen', 'HomeImprovement', 'Toys&Games', 'Car&Motorbike', 'Health&PersonalCare']
shape of text: (1465, 424)
shape of category: (1465, 9)


In [7]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Flatten, Reshape, SpatialDropout1D
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [8]:
#split data
X_train, X_test, Y_train, Y_test = train_test_split(text, category, test_size=0.2, random_state=42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

#apply Bidirectional LSTM
model = Sequential()
model.add(Embedding(2000, 128, input_length = 424))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(196, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(9,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

#Train the model for 5 epoches with embedding matrix you obtained earlier.
#print training loss and training accuracy for each epoch
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test,Y_test))  


(1172, 424) (1172, 9)
(293, 424) (293, 9)


2023-05-07 00:25:37.130287: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
37/37 [==============================] - 25s 576ms/step - loss: 1.3522 - accuracy: 0.3532 - val_loss: 1.1799 - val_accuracy: 0.3754
Epoch 2/10
37/37 [==============================] - 21s 573ms/step - loss: 0.7581 - accuracy: 0.6869 - val_loss: 0.5850 - val_accuracy: 0.8055
Epoch 3/10
37/37 [==============================] - 21s 572ms/step - loss: 0.4447 - accuracy: 0.8720 - val_loss: 0.4134 - val_accuracy: 0.8976
Epoch 4/10
37/37 [==============================] - 21s 570ms/step - loss: 0.2899 - accuracy: 0.9232 - val_loss: 0.3226 - val_accuracy: 0.9010
Epoch 5/10
37/37 [==============================] - 21s 574ms/step - loss: 0.2162 - accuracy: 0.9369 - val_loss: 0.2826 - val_accuracy: 0.9249
Epoch 6/10
37/37 [==============================] - 21s 572ms/step - loss: 0.1436 - accuracy: 0.9633 - val_loss: 0.2830 - val_accuracy: 0.9283
Epoch 7/10
37/37 [==============================] - 21s 573ms/step - loss: 0.0893 - accuracy: 0.9753 - val_loss: 0.2684 - val_accuracy: 0.9386

In [21]:
#auto category tagger function
#choose a random product
rand_idx = random.randint(0,len(df))
product_name = dfText['product_name'][rand_idx]
about_product = dfText['about_product'][rand_idx]

#reshape the tokenised input
input= np.reshape(text[rand_idx],(1, text[rand_idx].shape[0]))

#predict the category
pred = model.predict(input)
max_idx = np.argmax(pred)
predicted_category = category_list[max_idx]


def insert_newline_every_n_chars(text, n=10):
    new_text = ''
    for i, char in enumerate(text):
        new_text += char
        if (i + 1) % n == 0 and i != len(text) - 1:
            new_text += '\n'
    return new_text


product_name = insert_newline_every_n_chars(product_name, 60)
about_product = insert_newline_every_n_chars(about_product, 60)


#print out the results
actual_category = np.argmax(category[rand_idx])
print(f"Product name: {product_name}")
print("")
print(f"Product description: {about_product}")
print("")
print(f'Predicted Category: {predicted_category}')
print(f'Actual Category: {category_list[actual_category]}')

1/1 [==============================] - 0s 73ms/step
Product name: fire boltt visionary 1 78  amoled bluetooth calling smartwat
ch with 368 448 pixel resolution 100  sports mode  tws conne
ction  voice assistance  spo2   heart rate monitoring

Product description: fire boltt is india  no 1 wearable watch brand q122 by idc w
orldwide quarterly wearable device tracker q122  bluetooth c
alling watch   fire boltt visionary enables you to make and 
receive calls directly from your watch via the built in spea
ker and microphone  this smartwatch features a dial pad opti
on to access recent calls   sync your phone s contacts   1 7
8  amoled display    fire boltt visionary has a premium 368 
448 pixel resolution and 1 78  amoled display which comes wi
th the always on feature   over 100 sports modes    walking 
 running to tedious workout modes we have covered it all  fi
re boltt visionary smartwatch does a wonderful tracking job 
to each sports mode the user carries out in the day or at th
e gy